In [2]:
import numpy as np
import copy
import json
from message import *

ROS2 message(topic) from Perception node is in custermised `CAN.msg` format.\
In the example subscriber, topic from ROS2 has been transfered to `Message`(our data structure) format. CAN/Control team can directly use this Message data strucure.\
In this demo, since there is no message from ROS2, we read the fake results from json and transfer it to Message.

In [14]:
# Load example json file
msg_json = json.load(open("CAN_format_default.json"))

In [77]:
# msg_json

In [8]:
def json_to_message(msg, **kwargs):
    for k, v in kwargs.items():
        if isinstance(v, dict):
            setattr(msg, k, globals()[k](list(v.values()), CAN_KEYS[k]))
        elif isinstance(v, list):
            val = []
            for i in v:
                val.extend(list(i.values()))
            setattr(msg, k, globals()[k](val, CAN_KEYS[k]))

In [3]:
msg = Message()

In [10]:
json_to_message(msg, **msg_json)

`msg` is in the `Message` format, can print to vasualize

In [82]:
# print(msg)

`to_can()` function can transfer `Message` to a dict

In [84]:
# msg.to_can()

example of dbc key without track

In [92]:
msg.Objects

Objects(Rolling_Count=0.0, Current_Object_Count=5.0, Observation_Time_of_Hour=1800.0, dummy0=0.0, dummy1=0.0, dummy2=0.0, dummy3=0.0, dummy4=0.0, dummy5=0.0)

In [93]:
msg.Objects.Current_Object_Count

5.0

example of dbc key with track

In [94]:
msg.TrafficSign_TrackA

TrafficSign_TrackACollection(num_objects=2, data=[TrafficSign_TrackA(Rolling_Count=0.0, SignObjectID=1.0, Confidence=3.0, LongPos=32.11545181274414, Height_Above_Ground=2.6449999809265137, LatPos=-4.887977600097656, Sign_Type=3.0, Sign_Value=0.0, confidence_val=0.0, bbox_xmin=0.0, bbox_ymin=0.0, bbox_xmax=0.0, bbox_ymax=0.0, bbox3d_x=0.0, bbox3d_y=0.0, bbox3d_z=0.0, bbox3d_dx=0.0, bbox3d_dy=0.0, bbox3d_dz=0.0, bbox3d_ry=0.0, dummy0=0.0, dummy1=0.0, dummy2=0.0, dummy3=0.0, dummy4=0.0, dummy5=0.0), TrafficSign_TrackA(Rolling_Count=0.0, SignObjectID=2.0, Confidence=3.0, LongPos=32.11545181274414, Height_Above_Ground=2.6449999809265137, LatPos=8.433035850524902, Sign_Type=3.0, Sign_Value=0.0, confidence_val=0.0, bbox_xmin=0.0, bbox_ymin=0.0, bbox_xmax=0.0, bbox_ymax=0.0, bbox3d_x=0.0, bbox3d_y=0.0, bbox3d_z=0.0, bbox3d_dx=0.0, bbox3d_dy=0.0, bbox3d_dz=0.0, bbox3d_ry=0.0, dummy0=0.0, dummy1=0.0, dummy2=0.0, dummy3=0.0, dummy4=0.0, dummy5=0.0)])

In [96]:
msg.TrafficSign_TrackA[1]
# msg.TrafficSign_TrackA[1].SignObjectID

TrafficSign_TrackA(Rolling_Count=0.0, SignObjectID=2.0, Confidence=3.0, LongPos=32.11545181274414, Height_Above_Ground=2.6449999809265137, LatPos=8.433035850524902, Sign_Type=3.0, Sign_Value=0.0, confidence_val=0.0, bbox_xmin=0.0, bbox_ymin=0.0, bbox_xmax=0.0, bbox_ymax=0.0, bbox3d_x=0.0, bbox3d_y=0.0, bbox3d_z=0.0, bbox3d_dx=0.0, bbox3d_dy=0.0, bbox3d_dz=0.0, bbox3d_ry=0.0, dummy0=0.0, dummy1=0.0, dummy2=0.0, dummy3=0.0, dummy4=0.0, dummy5=0.0)

In [20]:
msg.get_avail_keys()
# msg.get_avail_ros_keys()

['RoadState']

Add new first level dbc key to `Message`

In [5]:
msg.add_new_key(V2XIntersection_MAP=[])

In [6]:
msg.get_avail_keys()

['V2XIntersection_MAP']

In [9]:
msg.V2XIntersection_MAP

V2XIntersection_MAP(Rolling_Count=0.0, IntersectionID=0.0, IngressLaneID=0.0, PlannedEgressLaneID=0.0, PlannedManeuver=0.0, SignalGroup=0.0, Map_Validity=0.0)

Two ways to assign value to second level keys

In [100]:
# 1
msg.AVState.GlobalAutonomyStatus = 1

In [1]:
msg.AVState

NameError: name 'msg' is not defined

In [12]:
# 2
setattr(msg.V2XIntersection_MAP, "IntersectionID", 2573)
setattr(msg.V2XIntersection_MAP, "IngressLaneID", 7)
setattr(msg.V2XIntersection_MAP, "PlannedEgressLaneID", 15)
setattr(msg.V2XIntersection_MAP, "PlannedManeuver", 2)
setattr(msg.V2XIntersection_MAP, "SignalGroup", 1)
setattr(msg.V2XIntersection_MAP, "Map_Validity", 1)

In [13]:
msg.V2XIntersection_MAP

V2XIntersection_MAP(Rolling_Count=0.0, IntersectionID=2573.0, IngressLaneID=7.0, PlannedEgressLaneID=15.0, PlannedManeuver=2.0, SignalGroup=1.0, Map_Validity=1.0)

Reset first level dbc key to default(all zero)

In [103]:
msg.reset_key(AVState=[])

In [104]:
msg.AVState

AVState(Rolling_Count=0.0, GlobalAutonomyStatus=0.0, SteeringCtrlActive=0.0, FrictionBrakeCtrlActive=0.0, PropulsionCtrlActive=0.0)

Be careful when copying, need deep copy to get a new `Message`

In [105]:
msg2 = msg

In [106]:
msg2.AVState

AVState(Rolling_Count=0.0, GlobalAutonomyStatus=0.0, SteeringCtrlActive=0.0, FrictionBrakeCtrlActive=0.0, PropulsionCtrlActive=0.0)

In [107]:
msg2.AVState.GlobalAutonomyStatus = 150

In [108]:
msg2.AVState

AVState(Rolling_Count=0.0, GlobalAutonomyStatus=150.0, SteeringCtrlActive=0.0, FrictionBrakeCtrlActive=0.0, PropulsionCtrlActive=0.0)

In [109]:
msg.AVState

AVState(Rolling_Count=0.0, GlobalAutonomyStatus=150.0, SteeringCtrlActive=0.0, FrictionBrakeCtrlActive=0.0, PropulsionCtrlActive=0.0)

In [110]:
msg2 = copy.deepcopy(msg)

In [111]:
msg2.AVState.GlobalAutonomyStatus = 1

In [66]:
msg.AVState

AVState(Rolling_Count=0.0, GlobalAutonomyStatus=150.0, SteeringCtrlActive=0.0, FrictionBrakeCtrlActive=0.0, PropulsionCtrlActive=0.0)

Some other useful functions

In [112]:
msg.Object_TrackA.get_keys()

['Rolling_Count',
 'ObjObjectID',
 'ObjObjectType',
 'LongPos',
 'LatPos',
 'RelLongVel',
 'RelLatVel',
 'Confidence',
 'Relative_Lane',
 'confidence_val',
 'bbox_xmin',
 'bbox_ymin',
 'bbox_xmax',
 'bbox_ymax',
 'bbox3d_x',
 'bbox3d_y',
 'bbox3d_z',
 'bbox3d_dx',
 'bbox3d_dy',
 'bbox3d_dz',
 'bbox3d_ry',
 'dummy0',
 'dummy1',
 'dummy2',
 'dummy3',
 'dummy4',
 'dummy5']

In [113]:
msg.Object_TrackA.get_num_keys()

27

to_can funciton transfer `Message` to dict, works for all levels of keys

In [114]:
msg.to_can()
msg.Object_TrackA.to_can()
msg.Object_TrackA[0].to_can()

{'Rolling_Count': 0.0,
 'ObjObjectID': 1.0,
 'ObjObjectType': 4.0,
 'LongPos': 7.984706,
 'LatPos': -2.4892166,
 'RelLongVel': 0.0,
 'RelLatVel': 1.0,
 'Confidence': 3.0,
 'Relative_Lane': 0.0,
 'confidence_val': 0.0,
 'bbox_xmin': 0.0,
 'bbox_ymin': 0.0,
 'bbox_xmax': 0.0,
 'bbox_ymax': 0.0,
 'bbox3d_x': 0.0,
 'bbox3d_y': 0.0,
 'bbox3d_z': 0.0,
 'bbox3d_dx': 0.0,
 'bbox3d_dy': 0.0,
 'bbox3d_dz': 0.0,
 'bbox3d_ry': 0.0,
 'dummy0': 0.0,
 'dummy1': 0.0,
 'dummy2': 0.0,
 'dummy3': 0.0,
 'dummy4': 0.0,
 'dummy5': 0.0}

to_ros funciton transfer `Message` to list, works for first/second levels of keys

In [70]:
# msg.to_ros(CAN) 
msg.Object_TrackA.to_ros()
msg.Object_TrackA[0].to_ros()

[0.0,
 1.0,
 4.0,
 7.984705924987793,
 -2.4892165660858154,
 0.0,
 1.0,
 3.0,
 0.0,
 0.0,
 1.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0]

In [116]:
# msg._to_ros()

In [117]:
msg.Object_TrackA.to_numpy()
msg.Object_TrackA[0].to_numpy()

array([ 0.       ,  1.       ,  4.       ,  7.984706 , -2.4892166,
        0.       ,  1.       ,  3.       ,  0.       ,  0.       ,
        0.       ,  0.       ,  0.       ,  0.       ,  0.       ,
        0.       ,  0.       ,  0.       ,  0.       ,  0.       ,
        0.       ,  0.       ,  0.       ,  0.       ,  0.       ,
        0.       ,  0.       ], dtype=float32)

In [75]:
msg_np = msg.Object_TrackA.to_numpy()
msg_np[:, 10:14]

array([[1., 0., 0., 0.],
       [1., 0., 0., 0.],
       [1., 0., 0., 0.],
       [1., 0., 0., 0.],
       [1., 0., 0., 0.]], dtype=float32)

In [37]:
msg_np[:, 10] = msg_np[:, 10] + 1
# msg_np

In [38]:
msg.reset_key(Object_TrackA=msg_np.flatten())

In [39]:
msg.Object_TrackA[0]

Object_TrackA(Rolling_Count=0.0, ObjObjectID=1.0, ObjObjectType=4.0, LongPos=7.984705924987793, LatPos=-2.4892165660858154, RelLongVel=0.0, RelLatVel=1.0, Confidence=3.0, Relative_Lane=0.0, confidence_val=0.0, bbox_xmin=1.0, bbox_ymin=0.0, bbox_xmax=0.0, bbox_ymax=0.0, bbox3d_x=0.0, bbox3d_y=0.0, bbox3d_z=0.0, bbox3d_dx=0.0, bbox3d_dy=0.0, bbox3d_dz=0.0, bbox3d_ry=0.0, dummy0=0.0, dummy1=0.0, dummy2=0.0, dummy3=0.0, dummy4=0.0, dummy5=0.0)